In [1]:
import pandas as pd
import re 

In [2]:
wellcome_apc_spend = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding ='latin1')
wellcome_apc_spend.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [3]:
wellcome_apc_spend.shape 

(2127, 5)

In [4]:
wellcome_apc_spend['Journal title'] = wellcome_apc_spend['Journal title'].str.lower()
wellcome_apc_spend.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,psychological medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [5]:
def get_pmcid(x):
    pmcid = re.findall('3\d{6}', str(x))
    if len(pmcid) > 0:
        return pmcid[- 1]

def get_pmid(x):
    pmid = re.findall('2\d{7}', str(x))
    if len(pmid) > 0:
        return pmid[0]
    
wellcome_apc_spend['PMID'] = wellcome_apc_spend['PMID/PMCID'].apply(get_pmid)
wellcome_apc_spend['PMCID'] = wellcome_apc_spend['PMID/PMCID'].apply(get_pmcid)

del wellcome_apc_spend['PMID/PMCID']

wellcome_apc_spend.drop_duplicates(subset='PMID', keep="last", inplace=True)
wellcome_apc_spend.drop_duplicates(subset='PMCID', keep="last", inplace=True)

wellcome_apc_spend.drop(wellcome_apc_spend[(wellcome_apc_spend.PMID == None) & (wellcome_apc_spend.PMCID == None)].index, inplace=True)

wellcome_apc_spend['GBPs'] = wellcome_apc_spend['COST (£) charged to Wellcome (inc VAT when charged)'].str.lstrip('£')
del wellcome_apc_spend['COST (£) charged to Wellcome (inc VAT when charged)']
wellcome_apc_spend['GBPs'] = pd.to_numeric(wellcome_apc_spend.GBPs, errors='coerce')

wellcome_apc_spend.dropna(subset=['GBPs'], inplace=True)
wellcome_apc_spend.drop(wellcome_apc_spend[(wellcome_apc_spend.GBPs > 10000.0)].index, inplace=True)

wellcome_apc_spend.head()

,Publisher,Journal title,Article title,PMID,PMCID,GBPs
2,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",23043264,3506128,642.56
3,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,23438330,3646402,669.64
4,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,23438216,3601604,685.88
7,ACS,mol pharm,Quantitative silencing of EGFP reporter gene b...,23057412,3495574,649.33
18,American Association of Immunologists,journal of immunology,Regulation of Foxp3+ Inducible Regulatory T Ce...,23455506,3607399,3108.08


In [6]:

wellcome_apc_spend['Journal title'].value_counts()[:5]

plos one                           22
journal of general virology         8
journal of biological chemistry     7
elsevier                            7
nucleic acids research              5
Name: Journal title, dtype: int64

In [7]:
round(wellcome_apc_spend.describe(),2)

,GBPs
count,401.00
mean,1972.59
std,750.55
min,270.00
25%,1549.44
50%,2040.00
75%,2355.52
max,5760.00


In [8]:
wellcome_apc_spend.groupby('Journal title').describe().head() 

GBPs                                                   \
                       count     mean std      min      25%      50%      75%   
Journal title                                                                   
acs chemical biology     1.0  1267.76 NaN  1267.76  1267.76  1267.76  1267.76   
acs nano                 1.0   693.39 NaN   693.39   693.39   693.39   693.39   
acta crystallography d   1.0   774.19 NaN   774.19   774.19   774.19   774.19   
acta diabetologica       1.0  2336.28 NaN  2336.28  2336.28  2336.28  2336.28   
acta physiol             1.0  1991.50 NaN  1991.50  1991.50  1991.50  1991.50   

                                 
                            max  
Journal title                    
acs chemical biology    1267.76  
acs nano                 693.39  
acta crystallography d   774.19  
acta diabetologica      2336.28  
acta physiol            1991.50